# `run_201905181215`

## Configuration

### `EstNLTK` version
```bash
(py36) ptammo@p12:~/Projects/cda-data-cleaning$ git -C ../estnltk show --summary
```
```
commit fd5b8314c31a227fb1135985c59ffe0d13aa1ece
Author: pault <pault@ut.ee>
Date:   Sat May 18 11:43:03 2019 +0300

    replaced re.Pattern with re.regex.Pattern in Vocabulary for compatibility with newer version of regex module
```

### `cda-data-cleaning` version
```bash
(py36) ptammo@p12:~/Projects/cda-data-cleaning$ git show --summary
```
```
commit f4b47bedf437511f1457b08c54ed3e33eefcdfe4
Author: pault <pault@ut.ee>
Date:   Fri May 17 23:06:15 2019 +0300

    #1120 update EventSegmentsTagger.output_attributes
```

### `egcut_epi_original.ini`
```
(py36) ptammo@p12:~/Projects/cda-data-cleaning$ cat configurations/egcut_epi_original.ini 
[database-configuration]
host = p12.stacc.ee
port = 5432
database_name = egcut_epi

username = ptammo
password = xxxxxx

role = egcut_epi_work_create
table_create_role = egcut_epi_work_create
table_read_role = egcut_epi_work_read

original_schema = original
work_schema = work


[luigi]
folder = luigi_targets


# LEGACY TO BE REMOVED
[classifications-database-configuration]
host = p12.stacc.ee
port = 5432
database_name = qs_rel
schema = rel_classifications
```

## Run `CreateTextsCollection`
```bash
(py36) ptammo@p12:~/Projects/cda-data-cleaning$ luigi --scheduler-port 8089 --module cda_data_cleaning CreateTextsCollection  --prefix=run_201905181215 --conf=egcut_epi_original.ini --workers=8
```

## Run `CreateEventsCollection`
```bash
(py36) ptammo@p12:~/Projects/cda-data-cleaning$ luigi --scheduler-port 8089 --module cda_data_cleaning CreateEventsCollection --prefix=run_201905181215 --conf=egcut_epi_original.ini --workers=1
```

## Results

In [1]:
from estnltk.storage import PostgresStorage

storage = PostgresStorage(pgpass_file='~/.pgpass',
                          dbname='egcut_epi',
                          schema='work',
                          role='egcut_epi_work_create')

INFO:storage.py:41: connecting to host: 'p12.stacc.ee', port: '5432', dbname: 'egcut_epi', user: 'ptammo'
INFO:storage.py:57: schema: 'work', temporary: False, role: 'egcut_epi_work_create'


### `run_201905181215_texts` collection

In [2]:
texts_collection = storage['run_201905181215_texts']
texts_collection

In [3]:
texts_collection.selected_layers = ['anonymised', 'event_tokens', 'event_headers', 'event_segments']

In [4]:
# find a text with an interesting `event_segments` layer
stop = False
for t in texts_collection:
    if len(t.event_segments) > 1:
        for span in t.event_segments:
            if span.DOCTOR:
                stop = True
                break
    if stop:
        break

In [5]:
t.event_segments

Layer(name='event_segments', attributes=('header', 'header_offset', 'DATE', 'HEADERWORD', 'DOCTOR', 'DOCTOR_CODE', 'SPECIALTY', 'SPECIALTY_CODE', 'ANONYM'), spans=SL[Span(start=0, end=644, text=' OPERATSIOONID \nOperatsioon 07.04.2011 14:01 Kirurg: PUUORG, EGON - D01460, E260 - ortopeedia. \n14:01 040208 SUURE TORULUU OSTEOTOOMIA \nZXE00 <= 1 tund \nNFK09 Reieluu fragmendi ekstsisioon \nZXD10 Plaaniline protseduur \nZXA05 Vasak pool Spinaalanesteesias pt. selili vasakul reiel sgutt, tehtud reie distaalsele otsale lateraalselt nahalõige, teravalt mindud teravalt vastus lateralise alt luuni. Avastatud periosti alune 8 cm pikkune ebakorrapärane selgelt piirdunud luuline moodustis reieluu metadiafüsaarsest tsoonist lateraalselt. See peitliga eemaldatud. Haav loputatud ja kihiti üksikõmblustega suletud. Side. \xa0 \n UURINGUD JA PROTSEDUURID '),
Span(start=669, end=930, text='WBC 8.91 (3,5 .. 8,8 E9/L )   \nRBC 5.08 (4,2 .. 5,7 E12/L )   \nHGB 153 (134 .. 170 g/L )   \nHCT 46 (40 .. 50 % )   \nMCV 89.6 (82 .. 98 fL )   \nMCH 30.1 (27 .. 33 pg )   \nMCHC 336 (317 .. 357 g/L )   \nPLT 236 (145 .. 390 E9/L )   \nRDW-CV 12.8 (11,6 .. 14 % )    '),
Span(start=942, end=1194, text='\nS,P-Glükoos 5.3 (mmol/L )  \nS,P-Uurea 3.6 (<8.3 mmol/L )  \nS,P-CRP <1 (<5 )  \nS,P-Na 140 (136 .. 145 mmol/L )  \nS,P-K 4.9 (3.4 .. 4.5 mmol/L )  \nGlomerulaarfiltratsiooni kiirus (eGFR) 83 (>90 mL/min/1.73m2 )  \nS,P-Kreatiniin 104 (62 .. 106 µmol/L )   '),
Span(start=1206, end=1436, text=' \nMärkus: Osakonnas määratud veregrupp 0 \nErütrotsütaarsete antikehade sõeluuring kahe erütrotsüüdiga Ei ole avastatud   \nAB0 ja RhD kinnitav määramine   \nAB0 veregrupp geeltehnikaga 0   \nRhD veregrupp geeltehnikaga Positiivne    '),
Span(start=1500, end=1784, text='Radioloogiline uuring, vastus nr 849_15803_1  \nKIRJELDUS: Vasak põlveliiges 2-s.  Põlveliigese piirkonnas reie- ja sääreluu metadiafüüside osas multiipelsed eksostoosid.  Pindluu lühenenud, tõenäoliselt esineb  luulne  sild pindluu-sääreluu vahel. Põlveliiges oluliste muutusteta.    '),
Span(start=1796, end=1832, text=' \nT. pallidum IgM+IgG Negatiivne    '),
Span(start=1844, end=1877, text='\nT. pallidum IgM+IgG Negatiivne \n')])

### `run_201905181215_events` collection

In [6]:
collection = storage['run_201905181215_events']
collection

,data type
texts_id,integer
epi_id,text
epi_type,text
schema,text
table,text
field,text
row_id,text
effective_time,timestamp without time zone
header,text
header_offset,integer


In [7]:
for id_, text, meta in collection.select(
    collection_meta=['texts_id', 'epi_id', 'epi_type', 'schema', 'table', 'field',
                     'row_id', 'effective_time', 'header', 'header_offset', 'event_offset'],
    layers=['anonymised']):

    if len(text.anonymised) > 1:
        break

In [8]:
meta

{'texts_id': 5,
 'epi_id': '1000808',
 'epi_type': 's',
 'schema': 'original',
 'table': 'procedures',
 'field': 'text',
 'row_id': '9',
 'effective_time': None,
 'header': '09.07.2009 13:27 - <ANONYM id="6" type="per" morph="_Y_ ?"/>, <ANONYM id="7" type="per" morph="_H_ sg n"/>',
 'header_offset': 1374,
 'event_offset': 1482}

In [9]:
text

Text(text=' - D04467 - Radioloog\nRadioloogiline uuring, vastus nr 14_15166_1\n\nKIRJELDUS: Paremal ACI algusosas põhiliselt pehmekoline naast, stenoos 59%. hemodünaamikat ei mõjuta. Vasakul ACI algusosas lubitihedaid naaste, senoosi maksimaalselt 30%. Vertebraalarterites vool tavapärane. \xa0 15.07.2009 14:28 - <ANONYM id="8" type="per" morph="_H_ sg n"/>, <ANONYM id="9" type="per" morph="_H_ sg n"/> - D04454 - E62 - radioloogia\nRadioloogiline uuring, vastus nr 612_15172_1\n\nKIRJELDUS: MRT angiograafia kaelaarteritest. Süstitud Omniscani 20,0.    Kaelaarterites hemodünaamiliselt olulisi stenoose nähtavale ei tule.  Paremal vertebraalarteri läbimõõt on asümmeetriliselt väiksem, alguskoht ei tule selgelt nähtavale- siin tõenäoliselt stenoos.  Vasak vertebraalarter iseärasusteta.  Ajuarterid Willise ringi piirkonnas  normipärast, sümmeetrilist laiust.   FLAIR uuringul bilat. periventrikulaarses valgeaines tõusnud signaaliga vaskulaarsele entsefalopaatiale viitavad alad. Paremal periventriklulaarsel ja vasemal basaaltuuamde piirkonnas likvoriseerunud lakunaarse infarkti kolded. ')

In [10]:
storage.close()